# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv from part 1
file = os.path.join('..', 'output_data', 'cities.csv')

# Read and display the dataframe
cities_df = pd.read_csv(file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,talakan,50.3167,130.3667,-21.64,82,100,3.74,RU,1612130312
1,hobart,-42.8794,147.3294,64.99,63,75,8.05,AU,1612130029
2,kodiak,57.7900,-152.4072,32.00,40,1,4.61,US,1612129864
3,kapaa,22.0752,-159.3190,75.20,73,40,14.97,US,1612130312
4,rikitea,-23.1203,-134.9692,79.11,62,35,7.20,PF,1612129869


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Set locations with lat and long from cities DataFrame
locations = cities_df[['Lat', 'Lng']]

#Set weight for gmap layer (humidity)
humidity = cities_df['Humidity']

# Plot gmaps with given info
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                                 max_intensity=100, point_radius=2
                                 )

# Add heat layer to gmaps
fig.add_layer(heat_layer)

#display gmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Find cities with max temps between 70 and 80
ideal_con = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70), :]

# Add to existing ideal DataFrame and find wind speeds below 10 mph
ideal_con = ideal_con.loc[ideal_con['Wind Speed'] < 10]

# Keep adding to the ideal DataFrame and find locations with zero cloudiness
ideal_con = ideal_con.loc[ideal_con['Cloudiness'] == 0]

# Display ideal DataFrame
ideal_con

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,kaeo,-35.1000,173.7833,75.00,49,0,7.00,NZ,1612130329
232,bandiagara,14.3501,-3.6104,78.80,22,0,6.91,ML,1612130343
254,saucillo,28.0167,-105.2833,70.54,22,0,7.56,MX,1612130346
278,wau,7.7029,27.9953,75.34,20,0,4.61,SS,1612130349
294,ahipara,-35.1667,173.1667,70.38,52,0,5.88,NZ,1612130351
311,geraldton,-28.7667,114.6000,78.80,25,0,8.05,AU,1612130353
317,gunjur,13.2019,-16.7339,78.80,44,0,6.69,GM,1612130354
342,sao filipe,14.8961,-24.4956,72.95,71,0,6.26,CV,1612130357
383,ouadda,8.0777,22.4007,71.31,25,0,5.41,CF,1612130362
396,moree,-29.4667,149.8500,75.20,64,0,8.05,AU,1612130364


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store ideal conditions df into new variable and add new column
hotel_df = ideal_con
hotel_df['Hotel Name'] = ''

# Display new dataframe with added column
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,kaeo,-35.1000,173.7833,75.00,49,0,7.00,NZ,1612130329,
232,bandiagara,14.3501,-3.6104,78.80,22,0,6.91,ML,1612130343,
254,saucillo,28.0167,-105.2833,70.54,22,0,7.56,MX,1612130346,
278,wau,7.7029,27.9953,75.34,20,0,4.61,SS,1612130349,
294,ahipara,-35.1667,173.1667,70.38,52,0,5.88,NZ,1612130351,
311,geraldton,-28.7667,114.6000,78.80,25,0,8.05,AU,1612130353,
317,gunjur,13.2019,-16.7339,78.80,44,0,6.69,GM,1612130354,
342,sao filipe,14.8961,-24.4956,72.95,71,0,6.26,CV,1612130357,
383,ouadda,8.0777,22.4007,71.31,25,0,5.41,CF,1612130362,
396,moree,-29.4667,149.8500,75.20,64,0,8.05,AU,1612130364,


In [ ]:
# Google maps example query search for reference when making params
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

In [7]:
# Create hotel names to add to and set query params
hotel_names = []
params = {"radius": 5000, 
          "type": "hotel",
          "keyword": "hotel",          
          "key": g_key}

# Loop through lat and long with iterrows to find nearest hotel
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    
    #set base url and response
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(url, params=params).json()
    
    #extract the results from API response
    results = response['results']
    try:
        hotel_names.append(results[0]['name'])
        print(f'The closest hotel at index {index} is: {results[0]["name"]}')
    except(IndexError):
        hotel_names.append('')
        print(f'Hotel was not found.. at index {index}: {lat}, {long}')

Hotel was not found.. at index 124: -35.1, 173.7833
The closest hotel at index 232 is: Hotel de la Falaise
The closest hotel at index 254 is: Hotel La Posada de San Judas Tadeo
The closest hotel at index 278 is: Amarula Lodge and Restaurant
The closest hotel at index 294 is: Ahipara Bay Motel
The closest hotel at index 311 is: Ocean Centre Hotel
The closest hotel at index 317 is: Dalaba Eco-Lodge
The closest hotel at index 342 is: Hotel Ocean View & Restaurante Seafood
Hotel was not found.. at index 383: 8.0777, 22.4007
The closest hotel at index 396 is: Artesian Spa Motel


In [8]:
# Add results from API response to hotel DataFrame..
hotel_df['Hotel Name'] = hotel_names
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,kaeo,-35.1000,173.7833,75.00,49,0,7.00,NZ,1612130329,
232,bandiagara,14.3501,-3.6104,78.80,22,0,6.91,ML,1612130343,Hotel de la Falaise
254,saucillo,28.0167,-105.2833,70.54,22,0,7.56,MX,1612130346,Hotel La Posada de San Judas Tadeo
278,wau,7.7029,27.9953,75.34,20,0,4.61,SS,1612130349,Amarula Lodge and Restaurant
294,ahipara,-35.1667,173.1667,70.38,52,0,5.88,NZ,1612130351,Ahipara Bay Motel
311,geraldton,-28.7667,114.6000,78.80,25,0,8.05,AU,1612130353,Ocean Centre Hotel
317,gunjur,13.2019,-16.7339,78.80,44,0,6.69,GM,1612130354,Dalaba Eco-Lodge
342,sao filipe,14.8961,-24.4956,72.95,71,0,6.26,CV,1612130357,Hotel Ocean View & Restaurante Seafood
383,ouadda,8.0777,22.4007,71.31,25,0,5.41,CF,1612130362,
396,moree,-29.4667,149.8500,75.20,64,0,8.05,AU,1612130364,Artesian Spa Motel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))